# TensorFlow Syntax

TensorFlow对不同的应用级别有着不同的搭建方法

In [1]:
import tensorflow as tf
import numpy as np

以MNIST作为训练用例

In [66]:
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 脚本形式构建Tensorflow张量流图

构建张量流图

In [64]:
# 内存->Tensor
x = tf.placeholder("float", shape=[None, 784])
x = tf.reshape(x,shape=[None,28,28,1])
y_ = tf.placeholder("float", shape=[None, 10])
W = tf.Variable(tf.truncated_normal([3,3,1,3], stddev=0.1))
b = tf.Variable(tf.constant(0.1, shape=[10]))
# 输出
y = tf.matmul(x, W) + b
# 交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y,labels = y_))
# 梯度下降
train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
#精度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#参数初始化
initializer = tf.global_variables_initializer()
#模型保存
saver = tf.train.Saver()

In [65]:
with tf.Session() as sess:
    initializer.run()
    for i in range(10):
        for _ in range(100):
            batch = mnist.train.next_batch(50)
            train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        print("Training accuracy :",accuracy.eval(feed_dict={x: batch[0], y_: batch[1]}))
        print("Validation accuracy :",accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print "Model saved in file: ", save_path

ValueError: Cannot feed value of shape (50,) for Tensor u'Placeholder_43:0', which has shape '(?, 10)'

In [56]:
with tf.Session() as sess:
    initializer.run()
    print("Validation accuracy :",accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

('Validation accuracy :', 0.134)


In [57]:
with tf.Session() as sess:
    initializer.run()
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    print("Validation accuracy :",accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Model restored.
('Validation accuracy :', 0.48930001)


## 函数形式构建Tensorflow张量流图

更复杂的网络需要复用节点，应函数化网络构建过程

tensorflow.get_variable() - 通过命名，使重复的调用不会覆盖原来的ops

In [61]:
def conv_relu(input, kernel_shape, bias_shape):
    weights = tf.get_variable("weights", kernel_shape,initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape,initializer=tf.constant_intializer(0.1))
    conv = tf.nn.conv2d(input, weights,strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

In [62]:
def MNIST_CLF(input_images):
    with tf.variable_scope("conv1"):
        relu1 = conv_relu(input_images, [3, 3, 32, 32], [32])
        print("relu1 :",relu1)
    with tf.variable_scope("conv2"):
        relu2 = conv_relu(relu1, [3, 3, 32, 32], [32])
        print("relu2 :",relu2)
        return relu2

In [ ]:
with tf.variable_scope("image_filters") as scope:
    scope.reuse_variables()
    result1 = MNIST_CLF(image1)
    result2 = MNIST_CLF(image2)